In [87]:
import pandas as pd
import numpy as np
import konlpy
from konlpy.tag import Hannanum
from konlpy.tag import Kkma
from konlpy.tag import Komoran
from konlpy.tag import Okt
import re
import random
import numpy as np

okt = Okt()
komoran=Komoran()
kkma=Kkma()
hannanum=Hannanum()

In [88]:
df = pd.read_excel("/Users/young/git/project/export/비전공자 개발자_2023-01-31.xlsx")
df.rename(columns={'Unnamed: 0':'Rank'},inplace=True)


In [89]:
df.tail(5)

,Rank,title,description,author,timestamp
1045,1045,"문과 개발자, 정말 비전공자도 IT 취업이 가능할까?","문과 개발자, 정말 비전공자도 IT 취업이 가능할까? 문과라서 취업할 길이 보이...",그대 미소에 봄이 돼줄게요,2021.09.18.
1046,1046,"자바 개발자 되는 법, 비전공자에게 적합한 과정은?",자바 개발자 되는 법 비전공자에게 맞는 과정 자바 개발자가 되기 위해 비전공자에...,IT교육및취업전문가 융쌤,2022.06.03.
1047,1047,"소프트웨어개발자되는법, 컴공출신 아닌 비전공자 자바웹포폴...",오늘은 컴공전공생이 아닌 비전공자가 #소프트웨어개발자되는법 #웹개발프로젝트 제작...,월요일에 만든 블로그,2022.01.20.
1048,1048,종로자바학원에서 비전공자도 개발자 취업 가능해요!!,종로자바학원에서 비전공자도 개발자 취업 가능해요!! 비전공자분들의 가장 큰 고민...,따봉아빠의 IT 일상,2017.09.08.
1049,1049,(비전공자 공부순서) 개발자가 하고 싶어요.....,그럼에도 불구하고 디지털노마드를 꿈꾸시거나 개발자의 꿈을 갖고 계신 분 혹은.....,초보개발자 _ 왕곰,2022.09.13.


In [90]:
df['structure'] = ''

In [91]:
def pos_seperator(pos):
    #품사
    pos = re.search(('(?<=\/)[a-zA-Z]*'),tup)[0]
    #형태소
    morphs = re.search(('.+(?=\/)'),tup)[0]

def add_scores():
    return 

def word_parser(df):
    morphs_list = []
    phrase_list = []
    structure_list = []
    # count_morphs_list = []
    # count_phrase_list = []
    # df 반복문 돌며 어절, 형태소 분해, 문장구조 분해 
    for idx,rows in df.iterrows():
        split_phrases = okt.phrases(rows[1])
        split_morphs = okt.pos(rows[1], join=True)

        for i in split_phrases:
            phrase_list.append(i + '/' + 'Noun')
        for i in split_morphs:
            if 'Punctuation' in i:
                continue
            morphs_list.append(i)
    #명사 중복 제거 후 구조화
        # tmp_structure = []
        tmp_structure = ''
        for jdx,tup in enumerate(okt.pos(rows[1])):
            if jdx > 0 and tmp == 'Noun' and tup[1] == 'Noun':
                continue
            tmp = okt.pos(rows[1])[jdx][1]
            # structure_list.append(tup[1])
            # tmp_structure += '_' + tup[1] if jdx != 0 or jdx != len(okt.pos(rows[1])) - 1 else tup[1]
            # if jdx != 0:
            #     tmp_structure += '_' + tup[1]
            # else:
            #     tmp_structure += tup[1]
            tmp_structure += '_' + tup[1]
            # structure_list.append([i for i in tup[1]])
        structure_list.append(tmp_structure)

    #unique값 찾기
    morphs_set = list(set(morphs_list))
    phrase_set = list(set(phrase_list))
    structure_set = list(set(structure_list))
    #빈도 카운트
    print(len(phrase_set))
    for i in range(len(morphs_set)):
        #두 리스트간 중복 삭제
        if morphs_set[i] in phrase_set:
            phrase_set.remove(morphs_set[i])
        morphs_set[i] = morphs_set[i] + '/' + str(morphs_list.count(morphs_set[i]))

    for i in range(len(phrase_set)):
        phrase_set[i] = phrase_set[i] + '_joint/' + str(phrase_list.count(phrase_set[i]))

    for i in range(len(structure_set)):
        structure_set[i] = structure_set[i] + '/' + str(structure_list.count(structure_set[i]))

    return phrase_set,morphs_set,structure_set,structure_list

phrase_set,morphs_set,structure_set,structure_list = word_parser(df)



3251


In [92]:
df_orgin_structure = []
for i in structure_list:
    
    split = re.split(('_'),i)
    del split[0]

    df_orgin_structure.append(split)

df['structure'] = df_orgin_structure



In [93]:
#단어 토큰 df화
frame = {
    'word' : [],
    'pos' : [],
    'cnt' : []
}
word_st = phrase_set + morphs_set
for i in word_st:
    try:
        split = re.split(('/'),i)
        # print(split[1],split[2],split[3])
        frame['word'].append(split[0])
        frame['pos'].append(split[1])
        frame['cnt'].append(int(split[2]))
    except:
        # print(i)
        continue

frame
df2 = pd.DataFrame(frame)
df2 = df2.sort_values(by="cnt",ascending=False)
df2

,word,pos,cnt
3321,비,Noun,1063
2961,전공자,Noun,1062
3677,개발자,Noun,1029
119,비전공자,Noun_joint,659
2657,취업,Noun,599
...,...,...,...
1442,부산IT아카데미 비전공자,Noun_joint,1
1444,고졸 개발자 취업 비전공자개발자,Noun_joint,1
1445,지방대,Noun_joint,1
1446,46회 SQL개발자,Noun_joint,1


In [94]:
# 문장구조 df화
structure_frame = {
    'structure' : [],
    'cnt' : []
}
for i in structure_set:
    split_cnt = re.split(('/'),i)
    cnt = int(split_cnt[-1])
    
    split = re.split(('_'),i)
    del split[0]
    split[-1] = re.split(('/'),split[-1])[0]
    # print(cnt)
    structure_frame['structure'].append(split)
    structure_frame['cnt'].append(cnt)
df3 = pd.DataFrame(structure_frame)
df3 = df3.sort_values(by="cnt",ascending=False)
df3

,structure,cnt
502,[Noun],40
297,"[Noun, Punctuation, Noun]",19
555,"[Noun, Josa, Noun]",18
228,"[Noun, Punctuation]",11
400,"[Noun, Punctuation, Noun, Punctuation]",9
...,...,...
282,"[Punctuation, Noun, Josa, Noun, Josa, Punctuat...",1
283,"[Noun, Adverb, Verb, Noun, Josa, Noun]",1
284,"[Punctuation, Noun, Punctuation, Noun, Josa, N...",1
285,"[Noun, Josa, Noun, Punctuation, Noun, Josa, No...",1


In [95]:
df3_pop = df3[df3['cnt'] > 1]

for i in range(1):
    sample_structure = random.choices(list(df3_pop['structure']),weights=[i for i in df3_pop['cnt']])[0]



In [96]:
sample_structure_dict = dict.fromkeys(sample_structure)

In [97]:
df2

,word,pos,cnt
3321,비,Noun,1063
2961,전공자,Noun,1062
3677,개발자,Noun,1029
119,비전공자,Noun_joint,659
2657,취업,Noun,599
...,...,...,...
1442,부산IT아카데미 비전공자,Noun_joint,1
1444,고졸 개발자 취업 비전공자개발자,Noun_joint,1
1445,지방대,Noun_joint,1
1446,46회 SQL개발자,Noun_joint,1


In [98]:
#문장구조와 단어 조합
def random_choice(sample_list, weight):
    sample = random.choices(sample_list,weights=weight)
    return sample[0]

for pos in sample_structure:
    if pos == 'Noun':
        coin_flip = random_choice(["Noun","Noun_joint"],[0.2,0.8])
        df2_n = df2[df2['pos'] == coin_flip]
    else:
        df2_n = df2[df2['pos'] == pos]
    df2_n.reset_index(drop=False)
    df2_n.sort_values(by='cnt',ascending=False)

    match_senetence = []
    for i,row in df.iterrows():
        if sample_structure == row[5]:
            match_senetence.append(row[1])

    words = []
    
    for i in range(10):
        if pos == 'Punctuation':
            sample = random_choice(["?","!","??","!!"],[1,1,1,1])

        else:
            sample = random_choice(list(df2_n['word']),[i for i in df2_n['cnt']])

        words.append(sample)
    sample_structure_dict[pos] = words

print(sample_structure)
print(match_senetence)
print(sample_structure_dict)

['Noun', 'Josa', 'Verb', 'Noun']
['실제로 받은 비전공자 개발자 면접 질문', '풀스택 개발자 비전공자 취업 루트로 꼽히는 이유']
{'Noun': ['컴퓨터', '교육', '프로그래밍', '비', '소프트웨어', '전공자', '학원', '전공자', '비', '개발자'], 'Josa': ['은', '한', '에서', '도', '인', '로', '도', '한', '만에', '가'], 'Verb': ['하는', '배우면서', '될', '해봤던', '대', '배웠어요', '받고', '할', '할', '하기']}


In [70]:
tester = list(df['title'])
# tester = ['자바개발자취업 비전공자 교육 과정 안내!']

word_key = []
word_joint = []
for i in tester:
    x = 0
    pos = okt.pos(i)
    # word_joint.append([j[0] for j in pos])
    while True:
        words = ''
        for j in range(x,len(pos)):
            if pos[j][1] == 'Punctuation':
                continue
            words += pos[j][0]
            # if j == x:
            #     word_key.append(words)
            #     continue
            # else:
            #     word_joint.append(words)
            if j != x:
                word_key.append(pos[x][0])
                word_joint.append(words)
        x += 1
        if x == len(pos):
            break;
    
    
word_joint_aggregate = dict.fromkeys(word_key)
df_following_word = pd.DataFrame(word_joint,index=word_key,columns=['word'])

df_unique_counter = pd.DataFrame(df_following_word['word'].value_counts())
df_unique_counter.rename(columns={"word":"count"},inplace=True)
df_unique_counter['word'] = df_unique_counter.index
df_following_word['key'] = df_following_word.index
df_following_word.drop_duplicates(inplace=True)
df_word_joint = pd.merge(df_following_word,df_unique_counter,left_on='word',right_on='word',how='inner')
df_word_joint = df_word_joint[['key','word','count']]

df_word_joint


,key,word,count
0,비,비전공자,1370
1,:,비전공자,1370
2,-,비전공자,1370
3,",",비전공자,1370
4,(,비전공자,1370
...,...,...,...
42823,),개발자가하고싶어요,2
42824,개발자,개발자가하고싶어요,2
42825,가,가하고,1
42826,가,가하고싶어요,1


In [116]:
#추천시스템
df_word_joint
pv_df = pd.pivot_table(df_word_joint,index=['word'],columns=['key'],values=['count'])
pv_df.columns = pv_df.columns.droplevel([0])
pv_df = pv_df.fillna(0)

In [185]:
df_test = df_word_joint[df_word_joint['key'] == '개발자']
df_test['odds'] = df_test['count']/df_test['count'].sum()

df_test = df_test.sort_values(by='odds',ascending=False)
list(df_test.head(5).iloc[:,1])
list(df_test.head(5).iloc[:,3])

/Users/young/opt/anaconda3/envs/test1/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


[0.044890162368672396,
 0.016077682266794013,
 0.01591849729385546,
 0.013053167780961477,
 0.00795924864692773]

In [242]:
key = '백엔드'

tmp = []
for i in range (15):

    print(key)
    df_test = df_word_joint[df_word_joint['key'] == key]
    if len(df_test['word']) == 0:
        key = okt.morphs(key)[-1]
        df_test = df_word_joint[df_word_joint['key'] == key]
        
    df_test['odds'] = df_test['count']/df_test['count'].sum()
    df_test = df_test.sort_values(by='odds',ascending=False)

    draw_words = list(df_test.head(5).iloc[:,1])
    draw_odds = list(df_test.head(5).iloc[:,3])
    draw = random_choice(draw_words,draw_odds)
    print(draw_words)

    print('draw=',draw)
    tmp.append(draw)

    key = draw.lstrip(key)

tmp

백엔드
['백엔드개발자', '백엔드개발자취업', '백엔드개발자비전공자', '백엔드개발자비', '백엔드개발자학원']
draw= 백엔드개발자
개발자
['개발자취업', '개발자비', '개발자비전공자', '개발자로', '개발자되는']
draw= 개발자취업
취업
['취업준비', '취업비전공자', '취업비', '취업을', '취업하는']
draw= 취업하는
하는
['하는방법', '하는법', '하는것', '하는이유', '하는비']
draw= 하는이유
이유
['이유와', '이유와연봉', '이유와연봉⭐', '이유feat', '이유feat패치']
draw= 이유와연봉⭐
와연봉⭐
['⭐비', '⭐비전공자', '⭐비전공자개발자', '⭐비전공자개발자취업', '⭐비전공자개발자취업을']
draw= ⭐비
비
['비전공자', '비전공자개발자', '비전공자도', '비전공자개발자취업', '비전공자가']
draw= 비전공자
전공자
['전공자개발자', '전공자도', '전공자개발자취업', '전공자가', '전공자도개발자']
draw= 전공자개발자취업
개발자취업
['취업준비', '취업비전공자', '취업비', '취업을', '취업하는']
draw= 취업준비
준비
['준비방법', '준비할', '준비하기', '준비하는', '준비해야']
draw= 준비하는
하는
['하는방법', '하는법', '하는것', '하는이유', '하는비']
draw= 하는방법
방법
['방법은', '방법에', '방법과', '방법비', '방법비전공자']
draw= 방법은
은
['은비전공자도', '은쉽지만', '은비전공자', '은쉽지만제대로된개발자되기', '은쉽지만제대로된개발자']
draw= 은쉽지만제대로된개발자되기
쉽지만제대로된개발자되기
['되기위해', '되기위', '되기위한', '되기프로젝트', '되기프로젝트생활코딩으로']
draw= 되기위
위
['위한', '위한개발자', '위한개발자취업', '위한공부', '위한개발자취업입문']
draw= 위한공부


/Users/young/opt/anaconda3/envs/test1/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


['백엔드개발자',
 '개발자취업',
 '취업하는',
 '하는이유',
 '이유와연봉⭐',
 '⭐비',
 '비전공자',
 '전공자개발자취업',
 '취업준비',
 '준비하는',
 '하는방법',
 '방법은',
 '은쉽지만제대로된개발자되기',
 '되기위',
 '위한공부']

In [240]:
a = ['백엔드개발자',
 '개발자비',
 '비전공자',
 '전공자개발자',
 '개발자비전공자',
 '전공자도',
 '도취업',
 '취업하는',
 '하는방법',
 '방법은',
 '은비전공자도',
 '도개발자',
 '개발자비전공자',
 '전공자가',
 '가될수']

 

,key,word,count
561,백엔드,백엔드개발자,59
564,백엔드,백엔드개발자취업,15
566,백엔드,백엔드개발자취업학원,1
1370,백엔드,백엔드개발자연봉,2
1371,백엔드,백엔드개발자연봉그리고,1
...,...,...,...
41649,백엔드,백엔드개발자교육비전공자취업,2
41650,백엔드,백엔드개발자교육비전공자취업전문,2
41651,백엔드,백엔드개발자교육비전공자취업전문학원,2
41652,백엔드,백엔드개발자교육비전공자취업전문학원추천,2
